In [1]:
import pandas as pd

df = pd.read_parquet('datasets/backup.parquet')

In [5]:
df.Photo_Url.unique().shape

(227889,)

In [8]:
import ray

ray.init()

2023-03-17 01:10:06,935	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.9
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


In [10]:
# download images to local folder
import requests
import os
from pathlib import Path
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

failed = []

# @ray.remote
def download_url(url, filename: Path):
    t0 = time.time()
    try:
        r = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(r.content)
        return(url, time.time() - t0)
    except Exception as e:
        print('Exception in download_url():', e)

# test 1 download
download_url(df.Photo_Url[0], Path('datasets/images/0.jpg'))


('https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/c95301eddca89cb2b6a5c5bfd991c9f93c4a35e3.jpg',
 0.36696386337280273)

In [11]:
# check exif
from PIL import Image
from PIL.ExifTags import TAGS

def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    raw_exif = image._getexif()
    if raw_exif is None:
        return {}
    exif = {}
    for tag, value in raw_exif.items():
        decoded = TAGS.get(tag, tag)
        exif[decoded] = value
    return exif

# test
get_exif('datasets/images/0.jpg')

{34665: 38, 274: 1, 40961: 65535, 40962: 1774, 40963: 2364}